In [ ]:
import csv
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline

In [ ]:
SEED = 42
DATA_PATH = r".\data"
MODEL_PATH = r".\models"

### Helper functions

In [ ]:
def write_data_file_paths():
    with open(rf"{DATA_PATH}\file_paths.csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["image", "mask", "label"])
        for sbj in range(100):
            if os.path.exists(rf"{DATA_PATH}\0\{sbj:02}"):
                label = 0
            elif os.path.exists(rf"{DATA_PATH}\1\{sbj:02}"):
                label = 1
            else:
                continue

            writer.writerow(
                [
                    rf"{sbj:02}\CT_partition.npy",
                    rf"{sbj:02}\CT_mask.npy",
                    label,
                ]
            )


write_data_file_paths()

In [ ]:
def load_haralick_features():
    try:
        data = pd.read_csv(rf"{DATA_PATH}\haralick.csv", index_col=0)
        labels = data.pop("y")
        return data, labels

    except FileNotFoundError:
        print("File not found, generating...")
        data, labels = [], []

        with open(rf"{DATA_PATH}\file_paths.csv", "r") as f:
            reader = csv.reader(f)
            next(reader)
            for image_path, mask_path, _ in reader:
                image = np.load(rf"{DATA_PATH}\{image_path}")
                mask = np.load(rf"{DATA_PATH}\{mask_path}")
                ...

## Scikit-Learn

In [ ]:
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    roc_auc_score,
)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC

In [ ]:
def print_metrics(y_true, y_pred, model_name) -> None:
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    print(model_name)
    print(f"Sensitivity: {tp / (tp + fn) * 100:.1f}%")
    print(f"Specificity: {tn / (tn + fp) * 100:.1f}%")
    print(f"Accuracy: {accuracy_score(y_true, y_pred) * 100:.1f}%")
    print(f"ROC-AUC: {roc_auc_score(y_true, y_pred) * 100:.1f}%")

    plt.imshow(cm, cmap=mpl.colormaps["Blues"])
    plt.colorbar()
    plt.title(f"{model_name} Confusion matrix")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.xticks([0, 1], ["BENIGN", "MALIGNANT"])
    plt.yticks([0, 1], ["BENIGN", "MALIGNANT"])
    for (j, i), label in np.ndenumerate(cm):
        color = "darkblue" if label < cm.max() / 2 else "white"
        plt.text(i, j, label, color=color)
    plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    *load_haralick_features(), test_size=0.2, random_state=SEED
)

svc_pipe = Pipeline(
    [
        ("scaler", MinMaxScaler()),
        # ("pca", PCA(0.9)),
        ("svc", SVC(kernel="linear", random_state=SEED)),
    ]
)

rf_pipe = Pipeline(
    [
        ("scaler", MinMaxScaler()),
        # ("pca", PCA(0.9)),
        ("rf", RandomForestClassifier(criterion="entropy", random_state=SEED)),
    ]
)

SVC

In [ ]:
svc_pipe.fit(X_train, y_train)
y_pred = svc_pipe.predict(X_test)
print_metrics(y_test, y_pred, "Linear SVC")

Random Forest

In [ ]:
rf_pipe.fit(X_train, y_train)
y_pred = rf_pipe.predict(X_test)
print_metrics(y_test, y_pred, "Random Forest")

## PyTorch

In [ ]:
import torch

from data import CTData
from unet import UNet

In [ ]:
IMAGE_DEPTH = 16
IMAGE_SIZE = 64
BATCH_SIZE = 8
EPOCHS = 2000

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

Load data

In [ ]:
def save_rois():
    with open(rf"{DATA_PATH}\file_paths.csv", "r") as f:
        reader = csv.reader(f)
        next(reader)
        
        for image, mask, label in reader:

            ct_scan = np.load(rf"{DATA_PATH}\{label}\{image}")
            ct_mask = np.load(rf"{DATA_PATH}\{label}\{mask}")

            roi = np.unique(np.where(ct_mask == 1), axis=1)
            roi_cx, roi_cy, roi_cz = (roi.max(axis=1) + roi.min(axis=1)) // 2
            bounding_box = ct_mask[
                roi_cx - IMAGE_DEPTH // 2 : roi_cx + IMAGE_DEPTH // 2,
                roi_cy - IMAGE_SIZE // 2 : roi_cy + IMAGE_SIZE // 2,
                roi_cz - IMAGE_SIZE // 2 : roi_cz + IMAGE_SIZE // 2,
            ]
            image_out = ct_scan[
                roi_cx - IMAGE_DEPTH // 2 : roi_cx + IMAGE_DEPTH // 2,
                roi_cy - IMAGE_SIZE // 2 : roi_cy + IMAGE_SIZE // 2,
                roi_cz - IMAGE_SIZE // 2 : roi_cz + IMAGE_SIZE // 2,
            ]

            os.mkdir(rf"{DATA_PATH}\rois\{image[:2]}")
            np.save(rf"{DATA_PATH}\rois\{image}", image_out)
            np.save(rf"{DATA_PATH}\rois\{mask}", bounding_box)


save_rois()

In [ ]:
images, masks, labels = [], [], []
with open(rf"{DATA_PATH}\file_paths.csv", "r") as f:
    reader = csv.reader(f)
    next(reader)

    for image, mask, label in reader:
        images.append(np.load(rf"{DATA_PATH}\rois\{image}"))
        masks.append(np.load(rf"{DATA_PATH}\rois\{mask}"))
        labels.append(int(label))

images = np.array(images)
masks = np.array(masks)
labels = np.array(labels)

In [ ]:
train_images, test_images, train_masks, test_masks, y_train, y_test = train_test_split(
    images, masks, labels, test_size=0.2, random_state=SEED
)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(
    CTData(train_images, train_masks), BATCH_SIZE, shuffle=True
)
test_dataloader = torch.utils.data.DataLoader(
    CTData(test_images, test_masks), 1, shuffle=False
)

In [ ]:
def threshold(data: torch.Tensor, level: float = 0.5) -> torch.Tensor:
    scaled = (data - data.min()) / (data.max() - data.min())
    scaled[scaled < level] = 0
    scaled[scaled >= level] = 1
    return scaled.type(torch.int32)


def pixel_accuracy(pred: torch.Tensor, true: torch.Tensor) -> torch.Tensor:
    return torch.sum(pred == true) / np.prod(pred.shape)


def iou(pred: torch.Tensor, true: torch.Tensor) -> torch.Tensor:
    intersection = torch.logical_and(true, pred)
    union = torch.logical_or(true, pred)
    return torch.sum(intersection) / torch.sum(union)


def dice_coeff(pred: torch.Tensor, true: torch.Tensor) -> torch.Tensor:
    intersection = torch.logical_and(true, pred)
    return 2 * torch.sum(intersection) / (torch.sum(true) + torch.sum(pred))

In [ ]:
model = UNet(residual=False, cat=False)
optimizer = torch.optim.Adam(model.parameters())
loss_fn = torch.nn.L1Loss()

In [ ]:
pixel_loss, iou_loss, dice_loss = [], [], []

for epoch in range(EPOCHS):
    model.train()
    model.to(device)
    for image, ct_mask in train_dataloader:
        image = image.to(device)
        ct_mask = ct_mask.to(device)
        
        optimizer.zero_grad()
        # pred = threshold(model(image))
        pred = model(image)
        loss = loss_fn(pred, ct_mask)
        loss.backward()
        optimizer.step()

    if epoch % 50 == 49:
        model.eval()
        model.cpu()
        epoch_pixel_loss, epoch_iou_loss, epoch_dice_loss = [], [], []
        for image, ct_mask in test_dataloader:
            # image = image.to(device)
            # mask = mask.to(device)
            pred = threshold(model(image))

            pixel = pixel_accuracy(pred, ct_mask)
            jaccard = iou(pred, ct_mask)
            dice = dice_coeff(pred, ct_mask).detach()

            epoch_pixel_loss.append(pixel)
            epoch_iou_loss.append(jaccard)
            epoch_dice_loss.append(dice)

        mean_pixel_loss = np.mean(epoch_pixel_loss)
        mean_iou_loss = np.mean(epoch_iou_loss)
        mean_dice_loss = np.mean(epoch_dice_loss)
        
        pixel_loss.append(mean_pixel_loss)
        iou_loss.append(mean_iou_loss)
        dice_loss.append(mean_dice_loss)

        print(f"Epoch {epoch + 1} / {EPOCHS} metrics:")
        print(f"PA {mean_pixel_loss:.4f} | IoU {mean_iou_loss:.4f} | Dice {mean_dice_loss:.4f}")

In [ ]:
torch.save(model.state_dict(), rf"{MODEL_PATH}\unet_l1_2000.pt")

In [ ]:
x = np.arange(50, EPOCHS + 1, 50)
plt.plot(x, pixel_loss, label="PA")
plt.plot(x, iou_loss, label="IoU")
plt.plot(x, dice_loss, label="Dice")
plt.legend()
plt.title("Test performance by epoch")
plt.show()

In [ ]:
model.cpu()
iter_data = iter(test_dataloader)

In [ ]:
image, ct_mask = next(iter_data)
pred = threshold(model(image), 0.75)

print(f"Pixel accuracy: {pixel_accuracy(pred, ct_mask):.4f}")
print(f"IOU (Jaccard): {iou(pred, ct_mask):.4f}")
print(f"Dice coefficient (F1-score): {dice_coeff(pred, ct_mask):.4f}")

image = image.cpu().detach().numpy()
pred = pred.cpu().detach().numpy()
ct_mask = ct_mask.cpu().detach().numpy()

fig, ax = plt.subplots(4, 12)
for slc in range(16):
    r, c = divmod(slc, 4)

    ax[r, c].imshow(image[0, 0, slc, :, :])
    ax[r, c + 4].imshow(pred[0, 0, slc, :, :])
    ax[r, c + 8].imshow(ct_mask[0, 0, slc, :, :])

    ax[r, c].axis("off")
    ax[r, c + 4].axis("off")
    ax[r, c + 8].axis("off")

## Full Pipeline

In [ ]:
import radiomics
import SimpleITK as sitk
from radiomics import featureextractor

radiomics.setVerbosity(40)

In [ ]:
def get_haralick_features(image: np.ndarray, mask: np.ndarray) -> np.ndarray:
    img = sitk.GetImageFromArray(image)
    msk = sitk.GetImageFromArray(mask)
    
    extractor = featureextractor.RadiomicsFeatureExtractor()
    extractor.disableAllFeatures()
    extractor.enableFeatureClassByName("glcm")
    result = extractor.execute(img, msk)
    return np.array([value for key, value in result.items() if "glcm" in key])

In [ ]:
full_dataloader = torch.utils.data.DataLoader(
    CTData(images, masks), 1, shuffle=False
)

In [ ]:
model = UNet(residual=False, cat=False)
model.load_state_dict(torch.load(rf"{MODEL_PATH}\unet_final.pt"))
model.eval()

In [ ]:
features = []
for image, _ in full_dataloader:
    ct_mask = threshold(model(image)).detach().numpy()
    image = image.detach().numpy()
    features.append(get_haralick_features(image[0, 0], ct_mask[0, 0]))
features = np.array(features)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, random_state=SEED
)

In [ ]:
svc_pipe.fit(X_train, y_train)
y_pred = svc_pipe.predict(X_test)
print_metrics(y_test, y_pred, "Linear SVC")

In [ ]:
import joblib
joblib.dump(svc_pipe, rf"{MODEL_PATH}\svc.joblib")

In [ ]:
image = np.load(rf"{DATA_PATH}\0\01\CT_partition.npy")
mask = np.load(rf"{DATA_PATH}\0\01\CT_mask.npy")
roi = np.unique(np.where(mask == 1), axis=1)
center, *_ = (roi.max(axis=1) + roi.min(axis=1)) // 2
center -= 8
print(center)

fig, ax = plt.subplots(4, 8)
for slc in range(16):
    r, c = divmod(slc, 4)

    ax[r, c].imshow(image[slc + center])
    ax[r, c + 4].imshow(mask[slc + center])

    ax[r, c].axis("off")
    ax[r, c + 4].axis("off")